# Stochastic Variational GP Regression with CIQ

## Overview

In this notebook, we'll give an overview of how to use SVGP stochastic variational regression ((https://arxiv.org/pdf/1411.2005.pdf)) to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. This is one of the more common use-cases of variational inference for GPs.

If you are unfamiliar with variational inference, we recommend the following resources:
- [Variational Inference: A Review for Statisticians](https://arxiv.org/abs/1601.00670) by David M. Blei, Alp Kucukelbir, Jon D. McAuliffe.
- [Scalable Variational Gaussian Process Classification](https://arxiv.org/abs/1411.2005) by James Hensman, Alex Matthews, Zoubin Ghahramani.

In [1]:
import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

# Make plots inline
%matplotlib inline

In [2]:
import urllib.request
import os
from scipy.io import loadmat
from math import floor


data = torch.Tensor(loadmat('../pol.mat')['data'])
X = data[:, :-1]
y = data[:, -1]

train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

x_mean = train_x.mean(dim=0)
x_stdv = train_x.std(dim=0)
good_dims = x_stdv.gt(0)
y_mean = train_y.mean()
y_stdv = train_y.std()

train_x = train_x.sub(x_mean).div(x_stdv)[..., good_dims]
test_x = test_x.sub(x_mean).div(x_stdv)[..., good_dims]
train_y = train_y.sub(y_mean).div(y_stdv)
test_y = test_y.sub(y_mean).div(y_stdv)

if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

## Creating a SVGP Model


For most variational/approximate GP models, you will need to construct the following GPyTorch objects:

1. A **GP Model** (`gpytorch.models.ApproximateGP`) -  This handles basic variational inference.
1. A **Variational distribution** (`gpytorch.variational._VariationalDistribution`) - This tells us what form the variational distribution q(u) should take.
1. A **Variational strategy** (`gpytorch.variational._VariationalStrategy`) - This tells us how to transform a distribution q(u) over the inducing point values to a distribution q(f) over the latent function values for some input x.

Here, we use a `VariationalStrategy` with `learn_inducing_points=True`, and a `CholeskyVariationalDistribution`. These are the most straightforward and common options.


#### The GP Model
  
The `ApproximateGP` model is GPyTorch's simplest approximate inference model. It approximates the true posterior with a distribution specified by a `VariationalDistribution`, which is most commonly some form of MultivariateNormal distribution. The model defines all the variational parameters that are needed, and keeps all of this information under the hood.

The components of a user built `ApproximateGP` model in GPyTorch are:

1. An `__init__` method that constructs a mean module, a kernel module, a variational distribution object and a variational strategy object. This method should also be responsible for construting whatever other modules might be necessary.

2. A `forward` method that takes in some $n \times d$ data `x` and returns a MultivariateNormal with the *prior* mean and covariance evaluated at `x`. In other words, we return the vector $\mu(x)$ and the $n \times n$ matrix $K_{xx}$ representing the prior mean and covariance matrix of the GP.

In [3]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import DeltaVariationalDistribution
from gpytorch.variational import CIQVariationalStrategy

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = DeltaVariationalDistribution(inducing_points.size(0))
        variational_strategy = CIQVariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=train_x.size(-1))
        )
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

inducing_points = torch.randn(2048, train_x.size(-1))
model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

### Training the Model

The cell below trains the model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalELBO` or ELBO), call backwards, and do a step of optimization.

In [4]:
from gpytorch.settings import record_ciq_stats as rcs
num_epochs = 10


model.train()
likelihood.train()

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

# Our loss object. We're using the VariationalELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

epochs_iter = tqdm.tqdm_notebook(range(num_epochs), desc="Epoch")
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=255, shuffle=True)

with gpytorch.settings.max_cg_iterations(200), gpytorch.settings.num_contour_quadrature(10):
    for i in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        minibatch_iter = tqdm.tqdm_notebook(train_loader, desc="Minibatch", leave=False)
        for x_batch, y_batch in minibatch_iter:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            loss.backward()
            optimizer.step()
            epochs_iter.set_postfix(
                loss=loss.item(),
                os=model.covar_module.outputscale.item(), noise=likelihood.noise.item(),
                res=rcs.minres_residual, mr_iter=rcs.num_minres_iter, ciq_d=rcs.ciq_diff
            )

### Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`). Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although this can be done by passing in minibatches of `test_x` rather than the full tensor.

In [6]:
test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

model.eval()
likelihood.eval()
nlls = []
mses = []
with torch.no_grad(), gpytorch.settings.max_cg_iterations(200), gpytorch.settings.num_contour_quadrature(63):
    for x_batch, y_batch in test_loader:
        preds = likelihood(model(x_batch))
        preds = torch.distributions.Normal(preds.mean.cpu(), preds.stddev.cpu())
        mses.append((preds.mean.cpu() - y_batch.cpu()).pow(2))
        nlls.append(-preds.log_prob(y_batch.cpu()))

nlls = torch.cat(nlls)
mses = torch.cat(mses)

print(f"Test NLL: {nlls.mean().item():.3f}, RMSE: {mses.mean().sqrt().item():.3f}")

Test NLL: 0.466, RMSE: 0.382
